In [1]:
from __future__ import division
import numpy as np
import os,sys
import pickle

from IPython.display import display, Image
from matplotlib import pyplot as plt

import Tools

from DataGenerator import genTrainValidFolds

%matplotlib inline

In [97]:
class FunModel():
    
    def __init__(self, x, y):
        self.trainX = x
        self.trainY = y
        self.batchSize = self.trainX.shape[0]
        
    def lm(self, W, x=np.array([])):
        # Not inclucing bias for now.
        if x.size:
            zt = np.dot(x,np.transpose(W))
        else:
            zt = np.dot(self.trainX, np.transpose(W))
        return zt
    
    def logitSoftmax(self, W, x=np.array([])):
        if x.size:
            zt = self.lm(W,x)
        else:
            zt = self.lm(W)
            
        out_h = np.exp(zt)
        return out_h / np.sum(out_h, axis=0)

    def lossCrossEntropy(self, W, x=np.array([]), y=np.array([])):
        noise = 1e-4
        if x.size:
#             print ('NOT EMPTY')
            out_h = self.logitSoftmax(W,x)
            avg_loss = np.sum(np.sum(-1 * y * np.log2(out_h + noise)))/x.shape[0]
        else:
#             print ('EMPTY')
            out_h = self.logitSoftmax(W)
            avg_loss = np.sum(np.sum(-1 * self.trainY * np.log2(out_h + noise)))/self.batchSize
            
        return avg_loss
#         print (avg_loss)
        
    def calNumericalGradient(self, W):
        """
            Esentially here we change each weight one after another and calculate the loss (each weight indicated to each cell in the weight 
            matrix of shape [10, 32*32]. Note after the loss for the a particular weight change is calculated, the weight is 
            put back to its oriinal value)
        
        """
        avg_loss = self.lossCrossEntropy(W)
        grad = np.zeros(W.shape)
        delta_h = 0.00001
        
        it = np.nditer(W, flags=['multi_index'], op_flags=['readwrite'])
        i = 0
        while not it.finished:
#             print ('pwpwpwpwpwpwpwp ', it.multi_index)
#             print ('asasasasasasasasasasa')
            i+=1
            ix = it.multi_index
            old_wght = W[ix]      # Get the Weight at Index 0,1 for first iteration
            W[ix] = old_wght + delta_h
            new_loss = self.lossCrossEntropy(W)   # Calculate the loss using the changed weight
            W[ix] = old_wght                # Put the weight back to its original value
#             print ('klklklkklkkkllklk', old_value)
            
            grad[ix] = (new_loss - avg_loss) / delta_h  # Compute the partial derivative
            it.iternext()
        print (i)
        return grad

In [ ]:
def main():
    for trainData, trainLabels, validData, validLabels in genTrainValidFolds():
        print ('Validation Data and Labels shape: ', validData.shape, validLabels.shape)
        print ('Training Data and Labels shape: ', trainData.shape, trainLabels.shape)
        
        # Generate a random W matrix
        W = np.random.rand(10, 32*32) * 0.001
        print (W.shape)
        print (trainData.shape)
        # create an object for the class KNN class
        obj_LMO = FunModel(trainData, trainLabels)
        # Calculate the loss using cross Entropy
#         obj_LMO.lossCrossEntropy(W, x=trainData, y=trainLabels)
        # Calculate the numerical gradient
        print (obj_LMO.calNumericalGradient(W))


        break
        
main()

Running i is : 0
Validation Data and Labels shape:  (5000, 1024) (5000, 10)
Training Data and Labels shape:  (45000, 1024) (45000, 10)
(10, 1024)
(45000, 1024)


In [25]:
a = np.array([[1, 2,3, 4],[5,6,4,5]])
b = sum(a)
print (a.shape, a)
print ()
print (b.shape,b)
c = np.array([[2, 2,2, 4],[5,6,2,1]])
print (a/b)
print (a*c)


(2, 4) [[1 2 3 4]
 [5 6 4 5]]

(4,) [6 8 7 9]
[[ 0.16666667  0.25        0.42857143  0.44444444]
 [ 0.83333333  0.75        0.57142857  0.55555556]]
[[ 2  4  6 16]
 [25 36  8  5]]
